In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
clientes = pd.read_csv('caso_estudo_clientes.csv', sep=';')
vendas = pd.read_csv('caso_estudo_vendas.csv', sep=';')
produtos = pd.read_csv('caso_estudo_produtos.csv', sep=';')
pagamentos = pd.read_csv('caso_estudo_pagamentos.csv', sep=';')
lojas = pd.read_csv('caso_estudo_lojas.csv', sep=';')

In [14]:
# === LIMPEZA ===

#Ajustando dados nulos
clientes.loc[clientes.nome.isnull(),'nome'] = 'Sem Nome'
clientes.loc[clientes.sexo.isnull(),'sexo'] = 'o'
clientes.loc[clientes.dt_nasc.isnull(),'dt_nasc'] = '1/1/2022'
clientes.loc[9,'valor'] = 3500
produtos.loc[9,'produto'] = 'Monitor Gamer'

#Ajustando Datas
clientes.dt_nasc = pd.to_datetime(clientes.dt_nasc, format='%m/%d/%Y')
vendas.dt_venda = pd.to_datetime(vendas.dt_venda, format='%d/%m/%Y')
pagamentos.dt_pgto = pd.to_datetime(pagamentos.dt_pgto, format='%d/%m/%Y')

#Índices
clientes = clientes.set_index('id')
vendas = vendas.set_index('id')
produtos = produtos.set_index('id')
pagamentos = pagamentos.set_index('id')
lojas = lojas.set_index('id')


In [15]:
# === FUSION ===

lista_geral = vendas.join(clientes.add_prefix('cliente_'), on='id_cliente')
lista_geral = lista_geral.join(produtos.add_prefix('produto_'), on='id_produto')
lista_geral = lista_geral.join(lojas.add_prefix('loja_'), on='id_loja')
lista_geral = lista_geral.join(pagamentos.set_index('id_venda'))

In [16]:
# === FEATURE ENGINEERING ===

# Compras sem pagamento
lista_geral['pagamento'] = 1
lista_geral.loc[lista_geral.dt_pgto.isnull(), 'pagamento'] = 0

# Tempo entre compra e pagamento
lista_geral['tempo_pagamento'] = (lista_geral.dt_pgto - lista_geral.dt_venda).dt.days

# Idade dos Clientes
lista_geral['cliente_idade'] = np.floor((pd.to_datetime('today') - lista_geral.cliente_dt_nasc)/np.timedelta64(1, 'Y'))

# Análise - parte 01

Perguntas para análise:
- Quais lojas mais vendem?
- Quais produtos mais vendem?
- Quais lojas geram maior receita?
- Quais produtos geram maior receita?

## Lojas que mais vendem

## Produtos que mais vendem

## Quais lojas geram maior receita?

## Quais produtos geram maior receita?